Takes in .AWD, sleep log if it exists, calendar, and spits out some pretty pictures

In [1]:
import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, '/data/MoodGroup/actigraphy/gavi/actigraPy') 

import actigraPy.actigraPy as act
import importlib
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta

/data/MoodGroup/actigraphy/gavi/actigraPy/actigraPy/actigraPy.py:23: MatplotlibDeprecationWarning: 
The matplotlib.backends.tkagg module was deprecated in Matplotlib 3.0 and will be removed in 3.2.
  import matplotlib.backends.tkagg as tkagg


If you want to run the test data, run this cell instead of the one below

In [3]:
# data and log directories
sub = '301' #subject NUMBER as STRING
sub_long = 'sub-MOA%s'%sub

data_dir = '/data/MoodGroup/actigraphy/KMOA/raw/%s'%sub_long #output directory
out_dir = '/home/shandlergs/output/%s'%sub_long

if not os.path.exists(out_dir):
    os.mkdir(out_dir)


## Subject and directory info

In [4]:
fn = {}
#.AWD
awd_fn = os.path.join(data_dir, '%s.AWD'%sub_long)
#sleeplog
fn['cal']= os.path.join(data_dir, '%s_calendar_log.xls'%sub_long)
#calendar
fn['log'] = os.path.join(data_dir, '%s_sleeplog.xls'%sub_long)
logs = []

if os.path.isfile(awd_fn):
    for name in fn.keys():
        if os.path.isfile(fn[name]):
            print("%s has %s file at %s"%(sub,name,fn[name]))
            logs.append(fn[name])
        else:
            print("no " + name)
            fn[name] = ''
else:
    print("DO NOT CONTINUE!! THERE IS NO AWD FILE")

301 has cal file at /data/MoodGroup/actigraphy/KMOA/raw/sub-MOA301/sub-MOA301_calendar_log.xls
no log


### Make Mtimes file

Figure out where to clip awd file

In [5]:
awd_dat = act.read_AWD(awd_fn)


## Now just run these cells in order to extract comments and write Mtimes

In [6]:
importlib.reload(act)
#Make master mk_idx
mk_idx=act.get_markers(awd_dat)

In [9]:
importlib.reload(act)
#Read logs
for name in fn.keys():
    if os.path.isfile(fn[name]):
        log_dat,kw_dat= act.read_log(fn[name],awd_dat)
        mk_idx[name]=log_dat['idx']
        


      OffDate   OffTime     OnDate    OnTime      Comment
0         NaT       NaN 2011-09-06  00:00:00        Start
1  2011-09-12  11:00:00 2011-09-12  13:00:00           7T
2  2011-09-12  15:00:00 2011-09-12  17:00:00          MEG
3  2011-09-12  21:30:00 2011-09-13  07:00:00  Sleep Study
4  2011-09-13  12:00:00 2011-09-13  14:30:00           3T
5  2011-09-14  21:30:00 2011-09-15  07:00:00  Sleep Study
6  2011-09-15  09:00:00 2011-09-15  09:40:00     Infusion
7  2011-09-15  15:00:00 2011-09-15  17:00:00          MEG
8  2011-09-15  21:30:00 2011-09-16  07:00:00  Sleep Study
9  2011-09-16  11:00:00 2011-09-16  13:00:00           7T
10 2011-09-17  12:00:00 2011-09-17  14:00:00           3T
11 2011-09-26  11:00:00 2011-09-26  13:00:00           7T
12 2011-09-26  15:00:00 2011-09-26  17:00:00          MEG
13 2011-09-26  21:30:00 2011-09-27  07:00:00  Sleep Study
14 2011-09-27  12:00:00 2011-09-27  14:30:00           3T
15 2011-09-28  21:30:00 2011-09-29  07:00:00  Sleep Study
16 2011-09-29 

Make passes their own marker

In [10]:
mk_idx['cal']

[(60538, 60658, '7T'),
 (60778, 60898, 'MEG'),
 (61168, 61738, 'Sleep Study'),
 (62038, 62188, '3T'),
 (64048, 64618, 'Sleep Study'),
 (64738, 64778, 'Infusion'),
 (65098, 65218, 'MEG')]

In [11]:
rm_idx=[]
for idx,block in enumerate(mk_idx['cal']):
    comment = block[2]
    if 'pass' in comment:
        rm_idx.append(idx)
rm_idx.sort(reverse=True)
mk_idx['pass']=[]
for idx in rm_idx:
    block = mk_idx['cal'].pop(idx)
    mk_idx['pass'].append(block)
mk_idx['pass']

[]

In [12]:
mk_idx['cal']

[(60538, 60658, '7T'),
 (60778, 60898, 'MEG'),
 (61168, 61738, 'Sleep Study'),
 (62038, 62188, '3T'),
 (64048, 64618, 'Sleep Study'),
 (64738, 64778, 'Infusion'),
 (65098, 65218, 'MEG')]

Write Mtimes

In [13]:
importlib.reload(act)
act.write_Mtimes(awd_dat,mk_idx,os.path.join(out_dir,sub_long))

,OnDate,OnTime,OffDate,OffTime,marker,Comment
0,01-Aug-11,10:59 AM,01-Aug-11,12:23 PM,z,
1,01-Aug-11,12:53 PM,01-Aug-11,01:47 PM,z,
2,01-Aug-11,01:51 PM,01-Aug-11,02:43 PM,z,
3,01-Aug-11,02:55 PM,01-Aug-11,04:21 PM,z,
4,01-Aug-11,05:15 PM,01-Aug-11,06:57 PM,z,
5,01-Aug-11,07:11 PM,01-Aug-11,07:59 PM,z,
6,01-Aug-11,08:05 PM,01-Aug-11,08:19 PM,z,
7,01-Aug-11,08:27 PM,01-Aug-11,08:51 PM,z,
8,01-Aug-11,08:57 PM,01-Aug-11,09:23 PM,z,
9,01-Aug-11,09:29 PM,01-Aug-11,10:31 PM,z,


## Clip AWD for graphs

In [14]:
#read AWD file
awd_dat = act.read_AWD(awd_fn)
idx={'start':[0],'end':[len(awd_dat['dt_list'])-1]}
#get the start and stops from each log if they exist
for log in logs:
    print(log)
    _, kw_dat = act.read_log(log,awd_dat)
    
    if 'watch_on' in kw_dat.keys():
        on_date = kw_dat['watch_on'].iloc[0]['OnDate']
        on_time = kw_dat['watch_on'].iloc[0]['OnTime']
        on = datetime(on_date.year,on_date.month,on_date.day,on_time.hour,on_time.minute)
        #check if that time is in dt_list:
        if awd_dat['dt_list'].count(on) > 0:
            on_idx=awd_dat['dt_list'].index(on)
            idx['start'].append(on_idx)

    if 'watch_off' in kw_dat.keys():
        off_date = kw_dat['watch_off'].iloc[0]['OffDate']
        off_time = kw_dat['watch_off'].iloc[0]['OffTime']
        off = datetime(off_date.year,off_date.month,off_date.day,off_time.hour,off_time.minute)
        if awd_dat['dt_list'].count(off) > 0:
            off_idx=awd_dat['dt_list'].index(off)
            idx['end'].append(off_idx)
 
idx

/data/MoodGroup/actigraphy/KMOA/raw/sub-MOA301/sub-MOA301_calendar_log.xls
      OffDate   OffTime     OnDate    OnTime      Comment
0         NaT       NaN 2011-09-06  00:00:00        Start
1  2011-09-12  11:00:00 2011-09-12  13:00:00           7T
2  2011-09-12  15:00:00 2011-09-12  17:00:00          MEG
3  2011-09-12  21:30:00 2011-09-13  07:00:00  Sleep Study
4  2011-09-13  12:00:00 2011-09-13  14:30:00           3T
5  2011-09-14  21:30:00 2011-09-15  07:00:00  Sleep Study
6  2011-09-15  09:00:00 2011-09-15  09:40:00     Infusion
7  2011-09-15  15:00:00 2011-09-15  17:00:00          MEG
8  2011-09-15  21:30:00 2011-09-16  07:00:00  Sleep Study
9  2011-09-16  11:00:00 2011-09-16  13:00:00           7T
10 2011-09-17  12:00:00 2011-09-17  14:00:00           3T
11 2011-09-26  11:00:00 2011-09-26  13:00:00           7T
12 2011-09-26  15:00:00 2011-09-26  17:00:00          MEG
13 2011-09-26  21:30:00 2011-09-27  07:00:00  Sleep Study
14 2011-09-27  12:00:00 2011-09-27  14:30:00           

{'start': [0, 51238], 'end': [65406]}

In [15]:
start = max(idx['start'])
end = min(idx['end'])
print('start = %d, end = %d'%(start,end))

start = 51238, end = 65406


## <font color='red'>Make sure the starts and ends make sense before clipping the data</font>


In [16]:
#put own number in start + end if disagree with above!
lim = [(awd_dat['DateTime'][start],awd_dat['DateTime'][end],"")]
clip_dat = act.clip_dat(lim,awd_dat)

In [17]:
clip_dat['DateTime'][-1]

'15-Sep-11 08:07 PM'

In [18]:
log_dat,kw_dat = act.read_log(os.path.join(out_dir,sub_long+'_Mtimes.csv'),clip_dat)

        OnDate    OnTime    OffDate   OffTime marker      Comment
0    01-Aug-11  10:59 AM  01-Aug-11  12:23 PM      z             
1    01-Aug-11  12:53 PM  01-Aug-11  01:47 PM      z             
2    01-Aug-11  01:51 PM  01-Aug-11  02:43 PM      z             
3    01-Aug-11  02:55 PM  01-Aug-11  04:21 PM      z             
4    01-Aug-11  05:15 PM  01-Aug-11  06:57 PM      z             
5    01-Aug-11  07:11 PM  01-Aug-11  07:59 PM      z             
6    01-Aug-11  08:05 PM  01-Aug-11  08:19 PM      z             
7    01-Aug-11  08:27 PM  01-Aug-11  08:51 PM      z             
8    01-Aug-11  08:57 PM  01-Aug-11  09:23 PM      z             
9    01-Aug-11  09:29 PM  01-Aug-11  10:31 PM      z             
10   01-Aug-11  10:41 PM  02-Aug-11  06:43 AM      z             
11   02-Aug-11  06:57 AM  02-Aug-11  08:13 AM      z             
12   02-Aug-11  08:33 AM  02-Aug-11  09:11 AM      z             
13   02-Aug-11  09:45 AM  02-Aug-11  10:23 AM      z             
14   02-Au

In [19]:
mk_idx=log_dat['mks']

## Make graphs

In [20]:
plots = {}
for name in fn.keys():
    if name in mk_idx.keys():
        plots[name]=mk_idx[name]
plots['pass']=mk_idx['pass']

KeyError: 'pass'

In [ ]:
plots

In [ ]:
importlib.reload(act)

act.plot_awd(clip_dat,plots,max_act=1000, show=False,fn_pref=os.path.join(out_dir,sub_long+'_M+logs'),plot_type='single',debug=True)


In [ ]:
importlib.reload(act)

act.plot_awd(clip_dat,plots,max_act=250,show=False,fn_pref=os.path.join(out_dir,sub_long+'_M+logs_zoom'),plot_type='single',debug=True)
